This IPython notebook trains the model with 3 convolutional layers, 4 densely connected layers including the final output softmax layer. Each of the 3 convolutional layers are followed by by a batch normalization layer and a maxpooling layer.
We use adam optimization during training.
This particular implementation uses data augmentation along with dropout for regularization optimization for training.

We generate additional 9 images for each training image. 

All the layers except the output layer use ReLU activation function to preserve the non-linearity in the data set.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend
from keras.callbacks import ModelCheckpoint
from numpy import unique, ndarray, mean, std
from matplotlib import pyplot as plt
from pandas import read_pickle
import numpy as np 

In [3]:
batch_size = 100
epochs = 20
vector_encoding = True
img_height, img_width = 48, 48
filepath = "adam-data-aug.h5"

In [4]:
data = read_pickle('dataset/input_data')

In [5]:
train = data.Training
validate = data.Validation
test = data.Testing

In [6]:
x_train, y_train = train
x_validate, y_validate = validate
x_test, y_test = test

In [7]:
def normalize(x):
    x = x - mean(x, axis = 1).reshape((-1,1))
    x = (x - mean(x, axis = 0)) / std(x, axis = 0)
    return x

In [8]:
x_train = normalize(x_train)
x_validate = normalize(x_validate)
x_test = normalize(x_test)

In [9]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0,
        horizontal_flip=True,
        fill_mode='wrap')

In [11]:
import itertools

new_x = []
new_y = []
for x,y in zip(x_train,y_train): 
    x = np.reshape(x, (1,48, 48,1))
    count = 8
#     i=1
    
    images_flow = datagen.flow(x, batch_size=1)
    for i, new_images in enumerate(images_flow):
        
        new_x.append(np.reshape(new_images[0], (2304,)))
        new_y.append(y)
        
        if i >= count:
            break

In [ ]:
np.asarray(new_x).shape

In [13]:
new_x = np.asarray(new_x)
new_y = np.asarray(new_y)

In [14]:
x_train = np.concatenate((x_train,new_x),axis=0)
y_train = np.concatenate((y_train,new_y),axis=0)

In [ ]:
np.asarray(x_train).shape

In [16]:
num_train_samples = x_train.shape[0]
num_validate_samples = x_validate.shape[0]
num_test_samples = x_test.shape[0]
num_labels = unique(y_train).shape[0]
if(vector_encoding):
    y_train = keras.utils.to_categorical(y_train, num_labels)
    y_validate = keras.utils.to_categorical(y_validate, num_labels)
    y_test = keras.utils.to_categorical(y_test, num_labels)

In [17]:
if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(num_train_samples, 1, img_height, img_width)
    x_validate = x_validate.reshape(num_validate_samples, 1, img_height, img_width)
    x_test = x_test.reshape(num_test_samples, 1, img_height, img_width)
    input_shape = (1, img_height, img_width)
else:
    x_train = x_train.reshape(num_train_samples, img_height, img_width, 1)
    x_validate = x_validate.reshape(num_validate_samples, img_height, img_width, 1)
    x_test = x_test.reshape(num_test_samples, img_height, img_width, 1)
    input_shape = (img_height, img_width, 1)

In [27]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_labels, activation='softmax'))

In [28]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='auto', 
                             period = 1)

In [ ]:
history = model.fit(x=x_train,
                    y=y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_validate, y_validate),
                    shuffle=True,
                    callbacks=[checkpoint])

In [ ]:
(loss, accuracy) = model.evaluate(x_test,
                                  y_test,
                                  batch_size=batch_size,
                                  verbose=1)
print("Loss: %.4f"%(loss))
print("Accuract: %.4f"%(accuracy))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylim(0,1)
plt.legend(['Train','Validation'], loc='lower right')
plt.savefig('Accuracy_adam_data_aug_batchnorm.png')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.ylim(0,2.5)
plt.legend(['Train','Validation'], loc='upper right')
plt.savefig('Loss_adam_data_aug_batchnorm.png')
plt.show()

In [ ]:
from keras.utils import plot_model
import pydot
import graphviz
import pydotplus

In [5]:
plot_model(model, to_file='adam_data_aug_batchnorm.png')